In [1]:
from utils import RL_with_constraint, RL_without_constraint, RLWKC_with_constraint, RLWKC_without_constraint, HTP, sparsePermutation, addNoise, meanNormalizedError, sgn, one_norm, only_priors
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from IPython.display import clear_output
import random  
import scipy
np.random.seed(90)
random.seed(90)

In [2]:
n_Permuted = 16                                           # number of permutation
cv_measurements = 5
m = 32                                                    # number of known correspondences
noise_fraction = 0.02
permutation_runs, noise_runs = 50, 50
p = 240                                                   # dimension of x
k = 14                                                    # sparsity of x
N_Metrics = np.zeros(shape = (permutation_runs, noise_runs, 9))
x = np.load("x_sparsity_" + str(k) + ".npy")
A_cv =  np.load("A_cv_fixed.npy")  
y_cv = addNoise(A_cv @ x, noise_fraction)     


for N in [90]:
    A = np.load("A_fixed.npy")[:N, :]           # extract N rows from the generated matrix

    for per_run in range(permutation_runs):
        print("N: {}, Permutation run: {}".format(N, per_run))
        P = sparsePermutation(N, m, n_Permuted)
        noiseLess_y = P @ A @ x
        z_true = noiseLess_y - A @ x
        max_abs_z = np.max(np.abs(z_true))
        sum_abs_z = np.sum(np.abs(z_true))

        l1_RLWKC_with_constraint, l2_RLWKC_with_constraint = None, None
        l1_RLWKC_without_constraint, l2_RLWKC_without_constraint = None, None
        l1_RL_with_constraint, l2_RL_with_constraint = None, None
        l1_RL_without_constraint, l2_RL_without_constraint = None, None
        l1_one_norm = None
        l1_only_priors = None

        for noise_run in range(noise_runs):
            print("N: {}, Noisy run: {}".format(k, noise_run))
            noisy_y = addNoise(noiseLess_y, noise_fraction)
            xhat_RLWKC_with_constraint, l1_RLWKC_with_constraint, l2_RLWKC_with_constraint = RLWKC_with_constraint(A, noisy_y, A_cv, y_cv, m, l1_RLWKC_with_constraint, l2_RLWKC_with_constraint)
            xhat_RLWKC_without_constraint, l1_RLWKC_without_constraint, l2_RLWKC_without_constraint = RLWKC_without_constraint(A, noisy_y, A_cv, y_cv, m, l1_RLWKC_without_constraint, l2_RLWKC_without_constraint)
            xhat_RL_with_constraint, l1_RL_with_constraint, l2_RL_with_constraint = RL_with_constraint(A, noisy_y, A_cv, y_cv, l1_RL_with_constraint, l2_RL_with_constraint)
            xhat_RL_without_constraint, l1_RL_without_constraint, l2_RL_without_constraint = RL_without_constraint(A, noisy_y, A_cv, y_cv, l1_RL_without_constraint, l2_RL_without_constraint)
            xhat_HTP = HTP(np.concatenate((A, A_cv), axis = 0), np.concatenate((noisy_y, y_cv), axis = 0), k)
            xhat_one_norm, l1_one_norm = one_norm(A, noisy_y, A_cv, y_cv, l1_one_norm)
            xhat_only_priors, l1_only_priors = only_priors(A, noisy_y, A_cv, y_cv, m, l1_only_priors)
  
            N_Metrics[per_run][noise_run][0] = meanNormalizedError(x, xhat_RLWKC_with_constraint)
            N_Metrics[per_run][noise_run][1] = meanNormalizedError(x, xhat_RLWKC_without_constraint)
            N_Metrics[per_run][noise_run][2] = meanNormalizedError(x, xhat_RL_with_constraint)
            N_Metrics[per_run][noise_run][3] = meanNormalizedError(x, xhat_RL_without_constraint)
            N_Metrics[per_run][noise_run][4] = meanNormalizedError(x, xhat_HTP)
            N_Metrics[per_run][noise_run][5] = meanNormalizedError(x, xhat_one_norm)
            N_Metrics[per_run][noise_run][6] = meanNormalizedError(x, xhat_only_priors)
            N_Metrics[per_run][noise_run][7] = max_abs_z
            N_Metrics[per_run][noise_run][8] = sum_abs_z
           

            print(N_Metrics[per_run][noise_run][0], N_Metrics[per_run][noise_run][1], N_Metrics[per_run][noise_run][2], N_Metrics[per_run][noise_run][3], N_Metrics[per_run][noise_run][4], N_Metrics[per_run][noise_run][5], N_Metrics[per_run][noise_run][6], N_Metrics[per_run][noise_run][7], N_Metrics[per_run][noise_run][8])

    np.save('N_Metrics' + str(N) + '.npy', N_Metrics)

N: 90, Permutation run: 0
N: 14, Noisy run: 0
Choose lambda1 properly
Choose lambda1 properly
Choose lambda1 properly
0.02766205898006779 0.03280248219158729 0.17394330337715835 0.1921847913400298 0.5881275668146787 0.5681997881445064 0.7228160481912573 9.442972307785281 46.221490230714664
N: 14, Noisy run: 1
Choose lambda1 properly
Choose lambda1 properly
Choose lambda1 properly
0.051216885124591154 0.05371821588826161 0.20254789197415135 0.1943576188124392 0.5980668921061587 0.5750970088532799 0.6591305117062204 9.442972307785281 46.221490230714664
N: 14, Noisy run: 2
Choose lambda1 properly
Choose lambda1 properly
Choose lambda1 properly
0.04369403647582028 0.04558332531770326 0.19962945286326966 0.17288694396261406 0.3546729083854239 0.572935598457543 0.6800525525228304 9.442972307785281 46.221490230714664
N: 14, Noisy run: 3
Choose lambda1 properly
Choose lambda1 properly
Choose lambda1 properly
0.04114371004092433 0.04513980280341486 0.20554200675482465 0.19809998487323321 0.6071

/Users/garweetsresth/miniconda3/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Choose lambda1 properly
0.036616793654890095 0.035419613524619115 0.16539186714326543 0.15803822004392798 0.6417460806358484 0.6547545400078326 0.7193942682668875 7.420845725281975 60.91784350629682
N: 14, Noisy run: 1
Choose lambda1 properly
0.050246302187751626 0.0541820719926371 0.18059253732551628 0.15842377677190772 0.6147376892138071 0.6517595636296697 0.8198000180248711 7.420845725281975 60.91784350629682
N: 14, Noisy run: 2
Choose lambda1 properly
0.029830189571717423 0.03815389595443322 0.17586058154632447 0.14467766937564064 0.6458879981934726 0.6439704811261693 0.7180001912143652 7.420845725281975 60.91784350629682
N: 14, Noisy run: 3
Choose lambda1 properly
0.03348094705348558 0.05005672082905465 0.17329472048948708 0.161202909571571 0.6359727928607376 0.6585629664489916 0.7555031983564369 7.420845725281975 60.91784350629682
N: 14, Noisy run: 4
Choose lambda1 properly
0.03305521219115321 0.04176093526142124 0.16189976692432037 0.15747983513900515 0.65481841935148 0.66095297